## 1️⃣ Clone Repository & Install Dependencies

In [6]:
# Clone the repository (or upload your files)
!git clone https://github.com/alpyaman/LLM_Fine-Tuning_Platform.git
%cd LLM_Fine-Tuning_Platform

# Or if you want to upload manually, uncomment:
# from google.colab import files
# print("Upload your project as a zip file")
# uploaded = files.upload()
# !unzip LLM_Fine-Tuning_Platform.zip
# %cd LLM_Fine-Tuning_Platform

Cloning into 'LLM_Fine-Tuning_Platform'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 60 (delta 17), reused 52 (delta 13), pack-reused 0 (from 0)
Receiving objects: 100% (60/60), 49.78 KiB | 8.30 MiB/s, done.
Resolving deltas: 100% (17/17), done.
/content/LLM_Fine-Tuning_Platform/LLM_Fine-Tuning_Platform


In [3]:
# Install all dependencies
%%capture
!pip install -r requirements.txt
!pip install nest-asyncio pyngrok  # Additional for Colab

In [4]:
# Verify GPU
import torch
print(f"🎮 CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🎯 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    print("⚠️ No GPU found! Enable GPU runtime.")

🎮 CUDA available: True
🎯 GPU: Tesla T4
💾 GPU Memory: 14.74 GB


## 2️⃣ Setup Redis (In-Memory Broker)

In [5]:
# Install and start Redis
%%capture
!apt-get install -y redis-server
!redis-server --daemonize yes

# Verify Redis is running
import time
time.sleep(2)
!redis-cli ping

## 3️⃣ Setup Environment Variables

In [7]:
import os
from google.colab import userdata

# Set HuggingFace token (add it to Colab Secrets: 🔑 icon on left)
# Or enter it directly here (not recommended for shared notebooks)
try:
    HF_TOKEN = userdata.get('HF_TOKEN')
except:
    HF_TOKEN = input("Enter your HuggingFace token: ")

# Set environment variables
os.environ['HF_TOKEN'] = HF_TOKEN
os.environ['REDIS_HOST'] = 'localhost'
os.environ['REDIS_PORT'] = '6379'
os.environ['STORAGE_TYPE'] = 'local'

print("✅ Environment configured")

✅ Environment configured


## 4️⃣ Create Test Dataset

In [8]:
# Create example dataset if it doesn't exist
import json

if not os.path.exists('example_data.jsonl'):
    example_data = [
        {"instruction": "What is machine learning?", "output": "Machine learning is a subset of artificial intelligence that enables computers to learn from data.", "input": ""},
        {"instruction": "Write a Python function to add two numbers", "output": "def add(a, b):\n    return a + b", "input": ""},
        {"instruction": "Translate to Spanish", "output": "Hola, ¿cómo estás?", "input": "Hello, how are you?"},
        {"instruction": "What is the capital of France?", "output": "The capital of France is Paris.", "input": ""},
        {"instruction": "Explain photosynthesis", "output": "Photosynthesis is the process by which plants convert sunlight into energy.", "input": ""}
    ]

    with open('example_data.jsonl', 'w') as f:
        for item in example_data:
            f.write(json.dumps(item) + '\n')

    print("✅ Created example_data.jsonl")
else:
    print("✅ example_data.jsonl already exists")

✅ example_data.jsonl already exists


## 5️⃣ Start Celery Worker in Background

In [ ]:
# Start Celery worker in background with better logging
import subprocess
import time
import os

# Kill any existing celery workers
!pkill -f "celery worker" || true
time.sleep(2)

print("🚀 Starting Celery worker with GPU support...")

# Create a log file for worker output
worker_log = open('/content/celery_worker.log', 'w')

# Start worker in background with explicit Python path
worker_process = subprocess.Popen(
    ['python', '-m', 'celery', '-A', 'phase2.celery_config', 'worker', 
     '--loglevel=info', '-Q', 'training', '--pool=solo'],
    stdout=worker_log,
    stderr=subprocess.STDOUT,
    cwd='/content/LLM_Fine-Tuning_Platform'
)

print(f"Worker PID: {worker_process.pid}")
print("⏳ Waiting for worker to initialize (15 seconds)...")
time.sleep(15)

# Check worker status
print("\n📋 Checking worker status...")
!celery -A phase2.celery_config inspect active

# Show recent logs
print("\n📜 Recent worker logs:")
!tail -n 30 /content/celery_worker.log

print("\n✅ Worker process running in background")
print("💡 To view full logs later, run: !cat /content/celery_worker.log")

^C
🚀 Starting Celery worker...
⚠️ Worker may not have started. Check logs below:
Error: No nodes replied within time constraint



## 6️⃣ Setup ngrok for Public API Access

In [10]:
# Setup ngrok to expose FastAPI publicly
from pyngrok import ngrok

# Optional: Set your ngrok auth token for better limits
# Get one free at: https://dashboard.ngrok.com/get-started/your-authtoken
try:
    NGROK_TOKEN = userdata.get('NGROK_TOKEN')
    ngrok.set_auth_token(NGROK_TOKEN)
except:
    print("⚠️ No ngrok token found. Using free tier (may have limits)")

# Kill any existing ngrok tunnels
ngrok.kill()

print("✅ ngrok configured")

✅ ngrok configured


## 7️⃣ Start FastAPI Server

In [11]:
# Run FastAPI with ngrok in background
import nest_asyncio
import uvicorn
from threading import Thread

# Allow nested event loops (required for Colab)
nest_asyncio.apply()

# Import FastAPI app
from phase2.api import app

# Start ngrok tunnel
public_url = ngrok.connect(8000, bind_tls=True)
print(f"\n{'='*60}")
print(f"🌐 Public API URL: {public_url}")
print(f"📚 API Docs: {public_url}/docs")
print(f"❤️ Health Check: {public_url}/health")
print(f"{'='*60}\n")

# Start FastAPI server in background thread
def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="info")

server_thread = Thread(target=run_server, daemon=True)
server_thread.start()

print("✅ FastAPI server started")
print("\n⚠️ Keep this cell running! The server will stop if you interrupt it.")


🌐 Public API URL: NgrokTunnel: "https://vinita-overgreedy-nonobstetrically.ngrok-free.dev" -> "http://localhost:8000"
📚 API Docs: NgrokTunnel: "https://vinita-overgreedy-nonobstetrically.ngrok-free.dev" -> "http://localhost:8000"/docs
❤️ Health Check: NgrokTunnel: "https://vinita-overgreedy-nonobstetrically.ngrok-free.dev" -> "http://localhost:8000"/health

✅ FastAPI server started

⚠️ Keep this cell running! The server will stop if you interrupt it.


## 8️⃣ Test the API

In [12]:
# Get the public URL
import requests
import time

# Wait for server to be ready
time.sleep(3)

# Get public URL from ngrok
tunnels = ngrok.get_tunnels()
if tunnels:
    API_URL = tunnels[0].public_url
    print(f"API URL: {API_URL}")

    # Test health endpoint
    try:
        response = requests.get(f"{API_URL}/health", timeout=10)
        print(f"\n✅ Health Check: {response.json()}")
    except Exception as e:
        print(f"⚠️ Health check failed: {e}")
else:
    print("⚠️ No ngrok tunnel found. Make sure the server is running.")

INFO:     Started server process [641]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


API URL: https://vinita-overgreedy-nonobstetrically.ngrok-free.dev
INFO:     35.197.51.144:0 - "GET /health HTTP/1.1" 200 OK

✅ Health Check: {'status': 'healthy', 'app': 'LLM Fine-Tuning Platform', 'version': '2.0.0', 'redis_connected': True, 'celery_workers': 0}


## 9️⃣ Upload Dataset

**Important Workflow:**
1. Upload dataset → Get `job_id`
2. Use the same `job_id` when starting training
3. This ensures the training job can find your uploaded dataset

In [13]:
# Upload the test dataset
with open('example_data.jsonl', 'rb') as f:
    files = {'file': ('example_data.jsonl', f, 'application/json')}
    response = requests.post(f"{API_URL}/upload", files=files)

if response.status_code == 200:
    result = response.json()
    print("✅ Dataset uploaded successfully!")
    print(f"Job ID: {result['job_id']}")
    print(f"Filename: {result['filename']}")

    # Save job_id for next step
    UPLOAD_JOB_ID = result['job_id']
else:
    print(f"❌ Upload failed: {response.text}")

INFO:     35.197.51.144:0 - "POST /upload HTTP/1.1" 200 OK
✅ Dataset uploaded successfully!
Job ID: c5552afc-657b-4c48-9d41-f28432516e2d
Filename: example_data.jsonl


## 🔟 Start Training Job

In [14]:
# Start a training job using the uploaded job_id
train_payload = {
    "dataset_filename": "example_data.jsonl",
    "job_id": UPLOAD_JOB_ID,  # ⚠️ IMPORTANT: Use the job_id from upload!
    "config": {
        "base_model": "unsloth/llama-3-8b-bnb-4bit",
        "max_steps": 20,  # Small number for testing
        "batch_size": 2,
        "learning_rate": 0.0002,
        "lora_r": 16
    }
}

response = requests.post(f"{API_URL}/train", json=train_payload)

if response.status_code == 200:
    result = response.json()
    print("✅ Training started successfully!")
    print(f"Job ID: {result['job_id']}")
    print(f"Status: {result['status']}")

    # Save for monitoring
    TRAIN_JOB_ID = result['job_id']
else:
    print(f"❌ Training failed to start: {response.json()}")

INFO:     35.197.51.144:0 - "POST /train HTTP/1.1" 200 OK
✅ Training started successfully!
Job ID: c5552afc-657b-4c48-9d41-f28432516e2d
Status: queued


## 1️⃣1️⃣ Monitor Training Progress

In [15]:
# Monitor training progress
import time
from IPython.display import clear_output

print(f"Monitoring job: {TRAIN_JOB_ID}\n")

while True:
    try:
        response = requests.get(f"{API_URL}/status/{TRAIN_JOB_ID}")

        if response.status_code == 200:
            status = response.json()

            clear_output(wait=True)
            print(f"Job ID: {TRAIN_JOB_ID}")
            print(f"Status: {status['status']}")

            if status.get('progress'):
                print(f"Progress: {status['progress']}%")

            if status.get('current_step') and status.get('total_steps'):
                print(f"Steps: {status['current_step']}/{status['total_steps']}")

            # Check if completed or failed
            if status['status'] == 'completed':
                print("\n✅ Training completed!")
                if status.get('result'):
                    result = status['result']
                    print(f"Duration: {result.get('training_duration_seconds', 0):.2f}s")
                    print(f"Output: {result.get('adapter_dir', 'N/A')}")
                break

            elif status['status'] == 'failed':
                print("\n❌ Training failed!")
                print(f"Error: {status.get('error', 'Unknown error')}")
                break

        time.sleep(5)  # Poll every 5 seconds

    except KeyboardInterrupt:
        print("\n⚠️ Monitoring stopped (job still running)")
        break
    except Exception as e:
        print(f"Error checking status: {e}")
        break

Job ID: c5552afc-657b-4c48-9d41-f28432516e2d
Status: queued
INFO:     35.197.51.144:0 - "GET /status/c5552afc-657b-4c48-9d41-f28432516e2d HTTP/1.1" 500 Internal Server Error
INFO:     35.197.51.144:0 - "GET /status/c5552afc-657b-4c48-9d41-f28432516e2d HTTP/1.1" 500 Internal Server Error
INFO:     35.197.51.144:0 - "GET /status/c5552afc-657b-4c48-9d41-f28432516e2d HTTP/1.1" 500 Internal Server Error
INFO:     35.197.51.144:0 - "GET /status/c5552afc-657b-4c48-9d41-f28432516e2d HTTP/1.1" 500 Internal Server Error
INFO:     35.197.51.144:0 - "GET /status/c5552afc-657b-4c48-9d41-f28432516e2d HTTP/1.1" 500 Internal Server Error
INFO:     35.197.51.144:0 - "GET /status/c5552afc-657b-4c48-9d41-f28432516e2d HTTP/1.1" 500 Internal Server Error
INFO:     35.197.51.144:0 - "GET /status/c5552afc-657b-4c48-9d41-f28432516e2d HTTP/1.1" 500 Internal Server Error
INFO:     35.197.51.144:0 - "GET /status/c5552afc-657b-4c48-9d41-f28432516e2d HTTP/1.1" 500 Internal Server Error
INFO:     35.197.51.144:0 - 

## 1️⃣2️⃣ Download Trained Model

In [16]:
# Download the trained model
response = requests.get(f"{API_URL}/download/{TRAIN_JOB_ID}")

if response.status_code == 200:
    result = response.json()
    print("✅ Model ready for download!")
    print(f"Location: {result['model_path']}")
    print(f"Files: {result['files']}")

    # Zip and download
    model_path = result['model_path']
    !zip -r trained_adapter.zip {model_path}

    from google.colab import files
    files.download('trained_adapter.zip')
    print("\n📥 Download started!")
else:
    print(f"❌ Download failed: {response.text}")

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

## 1️⃣3️⃣ API Information & Commands

In [ ]:
# Display API information
print("="*60)
print("🌐 Your API is Running!")
print("="*60)
print(f"\nPublic URL: {API_URL}")
print(f"API Docs: {API_URL}/docs")
print(f"Health: {API_URL}/health")
print("\n" + "="*60)
print("📡 cURL Examples:")
print("="*60)
print(f"""
# Upload dataset
curl -X POST "{API_URL}/upload" \\
  -F "file=@your_data.jsonl"

# Start training
curl -X POST "{API_URL}/train" \\
  -H "Content-Type: application/json" \\
  -d '{{
    "dataset_filename": "your_data.jsonl",
    "config": {{
      "max_steps": 100,
      "batch_size": 2
    }}
  }}'

# Check status
curl "{API_URL}/status/YOUR_JOB_ID"

# List all jobs
curl "{API_URL}/jobs"
""")
print("="*60)

## 🛠️ Utility: View Worker Logs

In [ ]:
# Check Celery worker status
!celery -A phase2.celery_config inspect active

## 🛠️ Utility: Stop Services

In [ ]:
# Stop all services
!pkill -f "celery worker"
ngrok.kill()
print("✅ All services stopped")

---

## 📝 Notes

### Important:
- ⚠️ **Keep cell 7 running** - FastAPI server runs there
- 🔒 **ngrok URL changes** each time you restart
- 💾 **Colab storage is temporary** - download your models!
- ⏱️ **Colab timeout** - Free tier disconnects after ~12 hours

### Tips:
1. **Save your work**: Download trained models immediately
2. **Use secrets**: Add tokens to Colab Secrets (🔑 icon)
3. **Monitor resources**: Check GPU usage with `!nvidia-smi`
4. **Longer training**: Increase `max_steps` for better results

### Troubleshooting:
- **Worker not starting**: Check Redis with `!redis-cli ping`
- **API not accessible**: Check ngrok tunnel in cell 7
- **Out of memory**: Reduce `batch_size` or `max_seq_length`
- **Import errors**: Rerun cell 2 to reinstall dependencies

### Resources:
- [Project Repository](https://github.com/YOUR_USERNAME/LLM_Fine-Tuning_Platform)
- [Phase 2 Guide](phase2/PHASE2_GUIDE.md)
- [Unsloth Documentation](https://github.com/unslothai/unsloth)

---

**🎉 Enjoy training your models in the cloud!**